# SoLU Circuits

## Imports

In [1]:
import pysvelte
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
import os
import pickle
from pathlib import Path
import time
import random
import tqdm.notebook as tqdm
import einops
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch
import plotly.graph_objects as go
from torch.utils.data import DataLoader
from functools import partial
import pandas as pd
import gc
import collections
import copy
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import transformers
from datasets import load_dataset
import json
from transformers import AutoTokenizer
import datasets
import wandb
from pprint import pprint
import math
import tqdm.auto as tqdm
from rich import print
from easy_transformer import EasyTransformer
from easy_transformer.hook_points import HookedRootModule, HookPoint
import sys
from os import path
import gdown

# Fix for pysvelte import peculiarities
sys.path.append('/home/user/.local/lib/python3.9/site-packages/pysvelte')